In [ ]:
import pandas as pd
import plotly.express as px

file = r"J:\ctgroup\Edward\DATA\VMI\20250303\Propylene Oxide 2W_calibrated\0_calibrated.h5"
df = pd.read_hdf(file, key='data')
df.head()

In [ ]:
import numpy as np
import sklearn

df_filtered = df[(df['raw_t'] > 200) & (df['raw_etof'] > 470) & (df['raw_t'] < 240) & (df['raw_etof'] < 510) & (
        df['px'] ** 2 + df['py'] ** 2 < 0.004)]
df_filtered['etof_diff'] = df_filtered['raw_etof'] - df_filtered['raw_t']
px.density_heatmap(df_filtered, x='raw_t', y="raw_etof", nbinsx=1000, nbinsy=1000, marginal_x="histogram",
                   marginal_y="histogram", width=800, height=800).show()
px.histogram(df_filtered, x='etof_diff', nbins=1000, width=800, height=800).show()

h2d, xedges, yedges = np.histogram2d(df_filtered['raw_t'], df_filtered['raw_etof'], bins=1000)
xv = 0.5 * (xedges[1:] + xedges[:-1])
yv = 0.5 * (yedges[1:] + yedges[:-1])

pts = np.vstack(df_filtered[['raw_t', 'raw_etof']].values)
principal_components = sklearn.decomposition.PCA(n_components=2).fit(pts)
direction = principal_components.components_[0]
mean = principal_components.mean_
t = np.linspace(-5, 5, 100)
line = mean[None, :] + t[:, None]

px.imshow(h2d.T, x=xv, y=yv, color_continuous_scale='Viridis', aspect='auto', origin='lower').update_layout(
        xaxis_title='raw_t',
        yaxis_title='raw_etof',
        width=800,
        height=800
).add_trace(
        px.line(x=line[:, 0], y=line[:, 1], color_discrete_sequence=['red'], line_shape='spline').data[0]
).show()